# Wave2Vec2
https://huggingface.co/docs/transformers/model_doc/wav2vec2

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch